This file contains the model for text training. It is based on the code from the hyperparameter tuning, in order to replicate the results obtained in hyperparam tuning as closely as possible - did not work!! completely different results.
Only the last layer is trained!! To train all layer remove the for loop setting require_grad to false (cell 4) and change model.classifier.parameters() to model.parameters() inside the optimizer (cell 4).

In [1]:
from functions_text_model import *
import os
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import json
from transformers import BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.utils.data import random_split
from sklearn.model_selection import ParameterGrid

c:\Users\celin\anaconda3\envs\DLproj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Function to encode the text
def encode_text(text):
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Input text
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attention masks
                        return_tensors = 'pt',     # Return pytorch tensors
                   )
    return encoded_dict['input_ids'], encoded_dict['attention_mask']

# PyTorch Dataset
class SarcasmDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        utterance = item['utterance']
        sarcasm = int(item['sarcasm'])
        input_ids, attention_mask = encode_text(utterance)
        return input_ids.flatten(), attention_mask.flatten(), sarcasm
# Create the DataLoader
# Load the data from the JSON file
with open('sarcasm_data.json') as f:
    data = json.load(f)

# Convert the data to a list of dictionaries
data = list(data.values())

dataset = SarcasmDataset(data)

In [3]:
# Set hyperparameters
learning_rate = 0.001
num_epochs = 5
batch_size = 16

In [6]:
torch.manual_seed(42)

# Set device
device = torch.device("cpu")

# Create a new model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

for parameter in model.bert.parameters():    # Remove if training the entire model
    parameter.requires_grad = False

model.to(device)

# Create a new optimizer with the current learning rate
optimizer = AdamW(model.classifier.parameters(), lr=learning_rate)
# Create the optimizer  
criterion = nn.CrossEntropyLoss()

metrics = {'ACC': acc, 'F1-weighted': f1}
# Define the size of the training set and the test set
train_size = int(0.8 * len(dataset))  # 80% of the data for training
test_size = len(dataset) - train_size  # 20% of the data for testing

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create the DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


# Train and evaluate the model for the current number of epochs
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    train_loss, train_metrics = train_epoch(model, optimizer, criterion, metrics, train_dataloader, device)
    eval_loss, eval_metrics = evaluate(model, criterion, metrics, test_dataloader, device)
    print('train metrics', train_metrics)
    print('test metrics', eval_metrics)


train Loss: 0.6737,  ACC: 0.5679, F1-weighted: 0.5379


100%|██████████| 9/9 [00:08<00:00,  1.04it/s]

eval Loss: 0.6881,  ACC: 0.5125, F1-weighted: 0.4808
train metrics {'ACC': 0.5678571428571428, 'F1-weighted': 0.5379149011116398}
test metrics {'ACC': 0.5125, 'F1-weighted': 0.4807867742852263}
